<a href="https://colab.research.google.com/github/khushi2912/Scheduling-Algorithms/blob/master/OS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#1 FCFS
#2 Priority Scheduling
#3 Round-Robin

def turnAroundTime(size,bt,wt,tat):
  
  for i in range(0,size):
    tat[i] = wt[i] + bt[i]




def fcfsWaitingTime(size,bt,at,wt):
  sumTime = [0] * size
  
  for i in range(1,size):

    sumTime[i] = sumTime[i-1] + bt[i-1]
    wt[i] = sumTime[i] - at[i]

    # when it arrives late and cpu is idle
    if(wt[i] < 0):
      wt[i] = 0



def fcfsAvgTime(size,bt,at):
  wt = [0] * size
  tat = [0] * size
  ct = [0] * size

  fcfsWaitingTime(size,bt,at,wt)
  turnAroundTime(size,bt,wt,tat)

  totalWT=0
  totalTAT=0

  for i in range(0,size):
    totalTAT = totalTAT + tat[i]
    totalWT = totalWT + wt[i]
    ct[i] = tat[i] + at[i]

  print("PID\t\tArrival Time\t\tBurst Time\t\tCompletion Time\t\tTurnaround Time\t\tWaiting Time")
  print("---------------------------------------------------------------------------------------------------------------------------------")
  for i in range(1,size+1):
    print("%d\t\t\t%d\t\t\t%d\t\t\t%d\t\t\t%d\t\t\t%d"%(i,at[i-1],bt[i-1],ct[i-1],tat[i-1],wt[i-1]))
 
  avgWT = totalWT/size
  avgTAT = totalTAT/size
  print("\nAverage waiting time is %s seconds."%avgWT)
  print("Average turnaround time is %s seconds."%avgTAT)
  


In [0]:
#2 Priority Scheduling - Non preemtive and AT for all 0
# sort and then fcfs
#https://www.geeksforgeeks.org/ways-sort-list-dictionaries-values-python-using-lambda-function/

def priorityScheduler(processes,size):
  processes = sorted(processes , key = lambda processes:processes[1] , reverse = True)
  at = [0] * size
  bt = [0] * size
  for i in processes:
    bt[i] = i[1]
  fcfsAvgTime(size,bt,at)

In [15]:

option = 0

print("-----------------------------") 
print("CPU Scheduling Algorithms: " )
print("-----------------------------") 
print("1. First Come First Serve") 
print("2. Non-Preemtive Priority Scheduling") 
print( "3. Round Robin")
print("4. ") 
print( "-----------------------------") 
print("Select > ",end=" ") 
option = int(input())
print("-----------------------------") 

if option is  1:
  print("here")
  fcfsAvgTime(3,[5,9,6],[0,3,6])

elif option is 2:
  print("2")
  priorityScheduler([],3)

elif option is 3:
  print("3")

elif option is 4:
  print("4")
else:
  print("Wrong Input")

-----------------------------
CPU Scheduling Algorithms: 
-----------------------------
1. First Come First Serve
2. Non-Preemtive Priority Scheduling
3. Round Robin
4. 
-----------------------------
Select >  1
-----------------------------
here
PID		Arrival Time		Burst Time		Completion Time		Turnaround Time		Waiting Time
---------------------------------------------------------------------------------------------------------------------------------
1			0			5			5			5			0
2			3			9			14			11			2
3			6			6			20			14			8

Average waiting time is 3.3333333333333335 seconds.
Average turnaround time is 10.0 seconds.
